**1. Import Libraries**

In [3]:

import pandas as pd
import plotly.graph_objects as go
import json

**2. Import and filter the dataset**

In [32]:
url = "https://www.dropbox.com/s/trdidwuhemik4qb/netflix_titles.csv?dl=1"
df = pd.read_csv(url)

In [33]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [34]:
# We need only title and description:
df[['title','description']]

,title,description
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm..."
1,Blood & Water,"After crossing paths at a party, a Cape Town t..."
2,Ganglands,To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,In a city of coaching centers known to train I...
...,...,...
8802,Zodiac,"A political cartoonist, a crime reporter and a..."
8803,Zombie Dumb,"While living alone in a spooky town, a young g..."
8804,Zombieland,Looking to survive in a world taken over by zo...
8805,Zoom,"Dragged from civilian life, a former superhero..."


**3. Preprocess the Data**

In [35]:
# Insert into new variable:
netflix_data = df.copy()

In [36]:
# Fill NaN values:
netflix_data['description'] = netflix_data['description'].fillna('')

**4. Prepare the ML model and the Data**

In [37]:
# Import model:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [38]:
# Fit the Data:
tfidf_matrix = tfidf.fit_transform(netflix_data['description'])

**5. Creating the Model :**

In [39]:
# Create the model :
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(netflix_data.index, index=netflix_data['title']).drop_duplicates()

# Create prediction function:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return netflix_data[['title','description']].iloc[movie_indices]

**6. Test the Model**

In [40]:
get_recommendations('Ganglands')

,title,description
424,Chhota Bheem: The Rise of Kirmada,Bheem and young lord Krishna team up to protec...
5113,Bright,"In an LA rife with interspecies tensions, a hu..."
6592,Den of Thieves,A highly skilled crew of bank robbers plotting...
5305,Narcos,The true story of Colombia's infamously violen...
3451,Kids on the Block,When their fun in the park is threatened by a ...
711,Security,After the assault of a young woman in their se...
1905,Cold Harbour,When a war breaks out between competing gangs ...
4031,Hajwala 2: Mysterious Mission,"In this sequel, a group of expert drivers skil..."
3976,The Eagle of El-Se'eed,A police officer and a drug lord become embroi...
7463,Miss Dynamite,"Wealthy, beautiful Valentina falls in love, on..."
